In [ ]:
##%matplotlib notebook
from ipywidgets import interact, widgets
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize as opt

# Echo State Networks 

In this section, we are going to move out attention to the problem of modeling (and estimating) **dynamic** nonlinear models, such as the one defined by nonlinear time series or different equations.

The tool we are going to discuss is the so call **Echo State Network (ESN)**, which in a nutshell is a class of nonlinear models with a state-space structure that is constructed entirely out of random functions - a bit like in the RWNN setup we just discussed.

## Constructing Echo State Networks

ESNs are constructed starting from the following general model equations:
$$ \begin{align}
x_t & = H(x_{t-1}, z_t), \\
y_t & = F(x_t),
\end{align}
$$
where here we use $x_t \in \mathbb{R}^{d_x}$, $y_t \in \mathbb{R}^{d_y}$ and $z_t \in \mathbb{R}^{d_z}$ to signify time series that do not necessarily need to be noisy. The sequence $\{ z_t \}_{t=1}^T$ is called the **inputs**, while sequence $\{ y_t \}_{t=1}^T$ is called the **targets** of the model.

An ESN proposes a specific form for (1)-(2), namely
$$ \begin{align}
x_t & = \alpha x_{t-1} + (1 -\alpha)\sigma(A x_{t-1} + C z_t + \zeta), \\
y_t & = W x_t, 
\end{align}
$$
where $A \in \mathbb{R}^{d_x \times d_x}$, $C \in \mathbb{R}^{d_x \times d_z}$ and $\zeta \in \mathbb{R}^{d_x}$ are *randomly drawn matrices/vectors*; $\alpha \in [0,1]$ is the so-called *leak-rate*, and $W \in \mathbb{R}^{d_y \times d_x}$ are the coefficients of the model that are actually estimated from data.

ESN models has show to be effective in reconstructing even complex systems and dynamics, and their expressive power is coupled with an increadible easy of implementation. Since all of the ESN parameters in equation (1) do not need estimation, we can generally proceed as follows:

1. Randomly draw $A$, $C$ and $\zeta$ from a chosen distribution, select $\alpha$.
2. Given inputs $\{ z_t \}_{t=1}^T$, *collect* the ESN states $\{ x_t \}_{t=1}^T$ by fixing an initial state $x_0$ and iterating
    $$ x_t = \alpha x_{t-1} + (1 -\alpha)\sigma(A x_{t-1} + C z_t + \zeta) .$$ 
3. Construct target matrix $Y := (y_1', \ldots, y_T')' \in \mathbb{R}^{T \times d_y}$ and state matrix $X := (x_1, \ldots, x_T)\in \mathbb{R}^{T \times d_x}$.
4. Use ridge regression with penalty $\lambda \geq 0$ to estimate 
    $$ \widehat{W} := (Z' Z + \lambda I)^{-1} Z'Y . $$

Let us write this in code:

In [ ]:
# DEFINE: state collection
def esn_collect(Z, A, C, zeta, alpha):
    T = Z.shape[0]
    Dx = A.shape[0]
    # NOTE: we are going to assume that x_0 == 0 for simplicity
    #       and that the nonlinear map \sigma(.) is tanh(.)
    X = np.zeros([T, Dx])
    X[0,:] = (1-alpha) * np.tanh(C @ Z[1,:] + zeta)
    for t in range(1,T):
        X[t,:] = alpha * X[t-1,:] + (1-alpha) * np.tanh(A @ X[t-1,:] + C @ Z[t,:] + zeta)
    return(X)

In [ ]:
# DEFINE: esn fitting
def esn_fit(Y, Z, L, A, C, zeta, alpha):
    # collect states
    X = esn_collect(Z, A, C, zeta, alpha)
     # ridge regression with penalty L
    # NOTE: we adjust here by T, the time length, so that if we increase
    #       the sample size, the penalization remains comparable
    T, K = X.shape
    W_hat = np.linalg.solve(np.dot(X.T, X) + (T*L)*np.eye(K), np.dot(X.T, Y))
    # compute fitted values and fit error
    Y_fit = X @ W_hat
    Error = Y - Y_fit
    return (W_hat, X, Y_fit, Error)

We can now try and fit an nonlinear deterministic sequence using the ESN model. For this example, let us consider a simple but not totally trivial model where
$$\begin{align}
z_t & = sin(t) , \\
y_t & = -0.3 + z_t + z_t^2 + 0.3\, cos(3 z_t - 3) . 
\end{align}
$$

In the following code, we randomly sample the entried in the parameter matrices $A$ and $C$ from a standard normal distribution, while $\zeta$ has entries from a uniform distribution over $[-1,1]$.
We are also able to tweak the ridge penalty, $\lambda$ (slider `Lambda`), and see how this affects the fit of the model.

In [ ]:
def esn_plot(T = 100, Dx = 10, rho = 0.5, gamma = 1, alpha = 0.1, Lambda = 1e-7):
    # generate data
    time = np.linspace(0, T*0.1, T)
    #print(time)
    Z = np.sin(time)[:,None]
    Y = Z + Z**2 - 0.3 + 0.3*np.cos(3*(Z - 1))

    # generate ESN parameters
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=0, scale=1, size=[Dx, Dx])
    C = rng.normal(loc=0, scale=1, size=[Dx, 1])
    zeta = rng.uniform(low=-1, high=1, size=Dx)

    # normalize
    A = (A / np.linalg.norm(A, 2)) * rho
    C = (C / np.linalg.norm(C, 2)) * gamma

    # fit esn
    _, X, Y_fit, _ = esn_fit(Y, Z, Lambda, A, C, zeta, alpha)

    fig = plt.figure()
    ax = fig.add_subplot(3, 1, 1)
    ax.plot(time, X)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN States")

    ax = fig.add_subplot(2, 1, 2)
    ax.plot(time, Z, c="0.6", zorder=1)
    ax.plot(time, Y, c="C1", zorder=2)
    ax.plot(time, Y_fit, c="C2", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN Fit")
    ax.legend(["$z_t$", "$y_t$", "ESN"])
    fig.canvas.draw()

interact(esn_plot, T=(1, 500), Dx=(5,50), rho=(0.1, 1.1), gamma=(0.1,3), alpha=(0.,1.),
            Lambda=widgets.FloatLogSlider(min=-8, max=2, value=1e-6));

Notice that, as we increase the dimension of state vector $x_t$ (changing slider `Dx`), we can see that the fit impoves to the point where it is hard to distinguish it from the true values of $y_t$.

## Autonomous ESN Models

One drawback from the previous ESN example is that often we do not only want to *fit* the data, but we would also like to *reconstruct the underlying dynamics*. That is, we would like the ESN to not only perfom well at a one-shot (one-step) forecasting task, but be able to reproduce the time series of interest directly.
We can ask this of the ESN model because it is state-space-based and, therefore, if we can iterate the states *autonomously* then we should also recover the true data-generating process.

The theory of ESNs fortunately guarantees that (in an appropriate sense) if the state space that $x_t$ belongs to is sufficiently big, then we will be reconstructing the underlying dynamics correctly. From a theory point of view, this is a very important result. And from the applied point of view, it is actually something we can easily implement.

Recall the ESN equations
$$ \begin{align}
x_t & = \alpha x_{t-1} + (1 -\alpha)\sigma(A x_{t-1} + C z_t + \zeta), \\
y_t & = W x_t, 
\end{align}
$$
and suppose now that we let $y_t = z_{t+1}$. That is, we impose a prective structure on our ESN model. This means that $y_{t-1} = z_t$.

Now, plug in equation (2) shifted to time $t-1$ - that is, $z_t = y_{t-1} = W x_{t-1}$ - into (1), and we obtain
$$ \begin{align*}
x_t & = \alpha x_{t-1} + (1 -\alpha)\sigma(A x_{t-1} + C z_t + \zeta) \\
    & = \alpha x_{t-1} + (1 -\alpha)\sigma(A x_{t-1} + C W x_{t-1} + \zeta) \\
    & = \alpha x_{t-1} + (1 -\alpha)\sigma\left((A + C W) x_{t-1} + \zeta\right) .
\end{align*} 
$$
Notice that the last line provides a recursion of the form $x_t = G(x_{t-1})$, where is $G(\cdot)$ is parametrized by $(A, C, \zeta, \alpha, W)$ and map $\sigma(\cdot)$. With this recursive formulation, which is often called the **autonomous** form of the ESN, we can predict $y_t$ (thus, $z_t$) at any future horizon, potentially:
$$ \begin{align*}
x_{t+h} & = G^h (x_t) =  \underbrace{G \circ G \circ \cdots \circ G}_{h \text{ times}}\,(x_t) \\
y_{t+h} & = W x_{t+h}, 
\end{align*}
$$
for any $h \geq 1$.

Let us implement the autonomous ESN iteration:

In [ ]:
# DEFINE: autonomous iteration
def esn_auto(X0, h, A, C, zeta, alpha, W):
    Dx = A.shape[0]
    Dy = W.shape[1]
    # pre-compute autonomous matrix
    R = A + C @ W.T
    # NOTE: we are going to assume that x_0 == 0 for simplicity
    #       and that the nonlinear map \sigma(.) is tanh(.)
    Xh = np.zeros([h, Dx])
    Yh = np.zeros([h, Dy])
    Xh[0,:] = X0
    Yh[0,:] = X0 @ W
    for t in range(1,h):
        Xh[t,:] = alpha * Xh[t-1,:] + (1-alpha) * np.tanh(R @ Xh[t-1,:] + zeta)
        Yh[t,:] = Xh[t,:] @ W
    return(Xh, Yh)

Considering again the deterministic time series model from the previous section, we now try and (1) fit process $\{y_t\}$ *directly* (i.e. it is both inputs and outputs); (2) forecast autonomously:

In [ ]:
def esn_plot(T = 100, h = 20, Dx = 10, rho = 0.5, gamma = 1, alpha = 0.1, Lambda = 1e-7):
    # generate data
    time = np.linspace(0, (T+h)*0.1, T+h)
    #print(time)
    Z = np.sin(time)[:,None]
    Y = Z + Z**2 - 0.3 + 0.3*np.cos(3*(Z - 1))

    # split train and test
    Y_train_in = Y[:T-1,:]
    Y_train_out = Y[1:T,:]
    Y_test = Y[T:,:]

    # generate ESN parameters
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=0, scale=1, size=[Dx, Dx])
    C = rng.normal(loc=0, scale=1, size=[Dx, 1])
    zeta = rng.uniform(low=-1, high=1, size=Dx)

    # normalize
    A = (A / np.linalg.norm(A, 2)) * rho
    C = (C / np.linalg.norm(C, 2)) * gamma

    # fit esn
    W_hat, X, Y_fit, _ = esn_fit(Y_train_out, Y_train_in, Lambda, A, C, zeta, alpha)

    # autonomous esn
    Xh, Yh = esn_auto(X[-1,:], h, A, C, zeta, alpha, W_hat)

    fig = plt.figure()
    ax = fig.add_subplot(3, 1, 1)
    ax.plot(time[0:T-1], X, c="0.6")
    ax.plot(time[T-1:-1], Xh)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN States")

    ax = fig.add_subplot(2, 1, 2)
    #ax.plot(time[0:T-1], Y_train_in, c="0.6", zorder=1)
    ax.plot(time[1:T], Y_train_out, c="0.6", zorder=2)
    ax.plot(time[1:T], Y_fit, c="k", zorder=3)
    ax.plot(time[T:], Y_test, c="C1", zorder=2)
    ax.plot(time[T:], Yh, c="C2", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN Fit & Autonomous")
    ax.legend(["$y_t$", "ESN Fit", "$y_t$ test", "ESN Auto"], loc="center left")
    fig.canvas.draw()

interact(esn_plot, T=(1, 500), h=(1, 100), Dx=(5,50), rho=(0.1, 1.1), gamma=(0.1,3), alpha=(0.,1.),
            Lambda=widgets.FloatLogSlider(min=-8, max=2, value=1e-6));

### 2.3 Predicting Chaotic Systems

ESNs have been widely applied to to prediction of nonlinear dynamical systems -- an area where they have been shown to be extremely effective. In this section, we will consider two cases

1. Autonomous ESN prediction of an ODE system - the *Lorenz attractor*.
2. Autonomous ESN prediction of a time-delay system - the *Mackey-Glass equations*.

The **Lorenz attractor** is a well-known three-dimensional system that exhibits caotic behavior. Its equations are:
$$\begin{align}
\frac{d x}{d t} & = \sigma(y - x) , \\
\frac{d y}{d t} & = x(\rho - z) - y , \\
\frac{d z}{d t} & = x y - \beta z .
\end{align}
$$

The following code provides a simple implementation of the Lorenz equations (with a plot):

In [ ]:
# DEFINE: Lorenz attractor
def lorenz_attractor(sigma, rho, beta, initial_state, delta_t, num_steps):
    """
    Simulate the Lorenz attractor.

    Parameters:
        sigma (float): Parameter sigma
        rho (float): Parameter rho
        beta (float): Parameter beta
        initial_state (tuple): Initial state (x0, y0, z0)
        num_steps (int): Number of simulation steps

    Returns:
        Tuple of NumPy arrays (x, y, z) representing the simulated trajectory.
    """
    x, y, z = initial_state
    x_vals, y_vals, z_vals = [x], [y], [z]

    for _ in range(num_steps):
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z

        x += delta_t * dx
        y += delta_t * dy
        z += delta_t * dz

        x_vals.append(x)
        y_vals.append(y)
        z_vals.append(z)

    return np.array(x_vals), np.array(y_vals), np.array(z_vals)

# Set the parameters
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0
initial_state = (1.0, 0.0, 0.0)

# Simulate the Lorenz attractor
x, y, z = lorenz_attractor(sigma, rho, beta, initial_state, 0.01, 10000)

# Plot the 3D trajectory
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, lw=0.5)
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('Lorenz Attractor')

plt.show()


Let us now try to autonomously predict the $y$ component of the Lorenz equations with an ESN:

In [ ]:
def esn_lorenz(steps = 1500, h = 500, Dx = 30, rho = 0.7, gamma = 1, alpha = 0.1, Lambda = 1e-9):
    # generate data (rescaled by 0.1x)
    data_x, data_y, data_z = lorenz_attractor(10.0, 28.0, 8.0/3, (1.0, 0.0, 0.0), 0.01, steps + h)
    data = np.column_stack([data_x, data_y, data_z]) / 10.0

    # split train and test
    Y_train_in = data[:steps-1,:]
    Y_train_out = data[1:steps,:]
    Y_test = data[steps:,:]

    # generate ESN parameters
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=0, scale=1, size=[Dx, Dx])
    C = rng.normal(loc=0, scale=1, size=[Dx, 3])
    zeta = rng.uniform(low=-1, high=1, size=Dx)

    # normalize
    A = (A / np.linalg.norm(A, 2)) * rho
    C = (C / np.linalg.norm(C, 2)) * gamma

    # fit esn
    W_hat, X, Y_fit, _ = esn_fit(Y_train_out, Y_train_in, Lambda, A, C, zeta, alpha)

    # autonomous esn
    Xh, Yh = esn_auto(X[-1,:], h, A, C, zeta, alpha, W_hat)

    fig = plt.figure()
    ax = fig.add_subplot(3, 1, 1)
    ax.plot(range(0, steps-1), X, c="0.6")
    ax.plot(range(steps-1, steps-1+h), Xh)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN States")

    ax = fig.add_subplot(2, 1, 2)
    ax.plot(range(1, steps), Y_train_out[:,1], c="0.6", zorder=2)
    ax.plot(range(1, steps), Y_fit[:,1], c="k", zorder=3)
    ax.plot(range(steps, steps+h+1), Y_test[:,1], c="C1", zorder=2)
    ax.plot(range(steps, steps+h), Yh[:,1], c="C2", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("Lorenz | Autonomous ESN")
    ax.legend(["$y_t$", "ESN Fit", "$y_t$ test", "ESN Auto"], loc="center left")
    fig.canvas.draw()

interact(esn_lorenz, steps=(500, 2500), h=(100, 1000), Dx=(5,50), rho=(0.1, 1.5), gamma=(0.1,3), alpha=(0.,1.),
            Lambda=widgets.FloatLogSlider(min=-9, max=2, value=1e-7));

Next, we try to forecast a more complex model. The Mackey-Glass equations are time-delay differency equations are such that the time derivative depends not only on the current value, but also on lagged values of the model.

We will consider the following Mackey-Glass model:
$$ \frac{d x(t)}{d t} = \frac{\beta x(t - \tau)}{1 + x(t - \tau)^n} - \gamma x(t) ,$$
where here the solution $x(t)$ is indexed by time explicitly to properly define delayed value $x(t - \tau)$. We are going to conside the original cofficients employed in the original paper, $\beta = 0.2$, $n = 10$ and $\gamma = 0.1$, and initial condition $x(0) = 0.5$.

The following function solves the Mackey-Glass model (plotting an example solution):

In [ ]:
# DEFINE: mackey-glass equations
def mackey_glass(beta, gamma, n, tau, a, delta_t, num_steps):
    """
    Simulate the Mackey-Glass equations.

    Parameters:
        beta (float): Parameter beta
        gamma (float): Parameter gamma
        n (float): Parameter n
        tau (float): Time delay parameter
        a (float): Amplitude parameter
        delta_t (float): Time step size
        num_steps (int): Number of simulation steps

    Returns:
        NumPy array representing the simulated trajectory.
    """
    x = np.zeros(num_steps)
    x[0] = 0.5  # Initial condition

    for t in range(1, num_steps):
        delay_index = max(t - int(tau / delta_t), 0)
        x[t] = x[t - 1] + delta_t * (beta * x[delay_index] / (1.0 + x[delay_index]**n) - gamma * x[t - 1])

    return a * x

# Simulate the Mackey-Glass equations
num_steps = 1000
trajectory = mackey_glass(0.2, 0.1, 10, 25, 0.1, 0.2, num_steps)

# Plot the simulated trajectory
plt.figure(figsize=(6,3))
plt.plot(trajectory)
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.title('Mackey-Glass')
plt.grid()
plt.show()


We can again try and iterate forward the Mackey-Glass solution by using an ESN autonomously:

In [ ]:
def esn_mackeyglass(steps = 1500, h = 400, Dx = 30, rho = 0.7, gamma = 1, alpha = 0.1, Lambda = 1e-9):
    # generate data (rescaled by 10x)
    data = mackey_glass(0.2, 0.1, 10, 25, 0.1, 0.2, steps + h)[:,None] * 10.0

    # split train and test
    Y_train_in = data[:steps-1,:]
    Y_train_out = data[1:steps,:]
    Y_test = data[steps:,:]

    # generate ESN parameters
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=0, scale=1, size=[Dx, Dx])
    C = rng.normal(loc=0, scale=1, size=[Dx, 1])
    zeta = rng.uniform(low=-1, high=1, size=Dx)

    # normalize
    A = (A / np.linalg.norm(A, 2)) * rho
    C = (C / np.linalg.norm(C, 2)) * gamma

    # fit esn
    W_hat, X, Y_fit, _ = esn_fit(Y_train_out, Y_train_in, Lambda, A, C, zeta, alpha)

    # autonomous esn
    Xh, Yh = esn_auto(X[-1,:], h, A, C, zeta, alpha, W_hat)

    fig = plt.figure()
    ax = fig.add_subplot(3, 1, 1)
    ax.plot(range(0, steps-1), X, c="0.6")
    ax.plot(range(steps-1, steps-1+h), Xh)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN States")

    ax = fig.add_subplot(2, 1, 2)
    ax.plot(range(1, steps), Y_train_out, c="0.6", zorder=2)
    ax.plot(range(1, steps), Y_fit, c="k", zorder=3)
    ax.plot(range(steps, steps+h), Y_test, c="C1", zorder=2)
    ax.plot(range(steps, steps+h), Yh, c="C2", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("Mackey-Glass | Autonomous ESN")
    ax.legend(["$y_t$", "ESN Fit", "$y_t$ test", "ESN Auto"], loc="center left")
    fig.canvas.draw()

interact(esn_mackeyglass, steps=(500, 2500), h=(100, 1000), Dx=(5,50), rho=(0.1, 1.5), gamma=(0.1,10), alpha=(0.,1.),
            Lambda=widgets.FloatLogSlider(min=-9, max=2, value=1e-9));

## Nonlinear Time Series

We now move from the deterministic setting to a stochastic one. In particular, we will try to forecast a simple nonlinear autoregressive time series model with an ESN model.

Let $\{\epsilon_t\}_{t \in \mathbb{Z}}$ be a sequence of independent random variables identically uniformly distributed over the interval $[-1, 1]$, and define
$$ X_t = \frac{2 X_{t-1}}{1 + 0.8\, X_{t-1}^2} + \epsilon_t .$$

In [ ]:
# DEFINE: nonlinear time series
def nonlin_ar(n, init=100, seed=12345):
    rng = np.random.default_rng(seed)
    E = np.zeros(n+init)
    X = np.zeros(n+init)
    E[0] = rng.uniform(low=-1, high=1)
    for t in range(1, n+init):
        E[t] = rng.uniform(low=-1, high=1)
        X[t] = 2 * X[t-1] / (1 + 0.8*X[t-1]**2) + E[t]
    return(X[init:], E[init:])

# Generate data
X_t, E_t = nonlin_ar(200)

# Plot the simulated trajectory
plt.figure(figsize=(6,3))
plt.plot(X_t)
plt.plot(X_t - E_t, c="C8")
plt.xlabel('Time Index')
plt.ylabel('Value')
plt.title('Nonlinear AR(1)')
plt.legend(["$x_t$", "$x_t - \epsilon_t$"], loc="upper left")
plt.grid()
plt.show()

In [ ]:
def nonlin_ar_condpred(h, X0, mc=1000, seed=54321):
    rng = np.random.default_rng(seed)
    Xh = np.zeros([h, mc])
    Xh[0,:] = X0
    for t in range(1, h):
        # randomly sample innovations
        E = rng.uniform(low=-1, high=1, size=mc)
        Xh[t,:] = 2 * Xh[t-1,:] / (1 + 0.8*Xh[t-1,:]**2) + E
    return(Xh)

# Simulate best conditional predictor 
h = 30
condpred = nonlin_ar_condpred(h, X_t[-1], mc=50000)

# Plot (with prediction bands)
fig = plt.figure(figsize=(6,3))
ax = fig.add_subplot(1, 1, 1)
ax.plot(range(-19, 1), X_t[-20:], c="0.6", zorder=1)
ax.plot(range(1, h), np.mean(condpred[1:,], axis=1), c="C1", zorder=3)
ax.fill_between(range(1, h), *np.quantile(condpred[1:,], [0.05, 0.95], axis=1), color="C1", alpha=0.2)
ax.set_axisbelow(True)
ax.grid()
ax.set_title("Nonlinear Regression")
ax.legend(["$x_t$"], loc="upper left")
fig.canvas.draw()

In [ ]:
def esn_nonlinear_ar(steps = 200, h = 30, Dx = 30, rho = 0.7, gamma = 1, theta = 1, alpha = 0.1, Lambda = 1e-6):
    # generate data (rescaled by 10x)
    X_t, E_t = nonlin_ar(steps+h)
    data = X_t[:,None]

    # split train and test
    X_t_train_in = data[:steps-1,:]
    X_t_train_out = data[1:steps,:]
    #X_t_test = data[steps:,:]

    # generate ESN parameters
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=0, scale=1, size=[Dx, Dx])
    C = rng.normal(loc=0, scale=1, size=[Dx, 1])
    zeta = rng.uniform(low=-1, high=1, size=Dx)

    # normalize
    A = (A / np.linalg.norm(A, 2)) * rho
    C = (C / np.linalg.norm(C, 2)) * gamma
    zeta = theta * zeta

    # fit esn
    W_hat, states, X_t_fit, _ = esn_fit(X_t_train_out, X_t_train_in, Lambda, A, C, zeta, alpha)

    # autonomous esn
    Xh, Yh = esn_auto(states[-1,:], h, A, C, zeta, alpha, W_hat)

    # best conditional predictor 
    condpred = nonlin_ar_condpred(h+1, X_t_train_out[-1,:], mc=50000)

    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(3, 1, 1)
    ax.plot(range(0, steps-1), states, c="0.6")
    ax.plot(range(steps-1, steps-1+h), Xh)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("ESN States")

    ax = fig.add_subplot(2, 1, 2)
    ax.plot(range(1, steps+h), X_t[1:,], c="C0", alpha=0.4, zorder=2)
    ax.plot(range(1, steps+h), X_t[1:,] - E_t[1:], c="C8", zorder=2)
    ax.plot(range(1, steps), X_t_fit, c="k", zorder=3)
    ax.plot(range(steps, steps+h), np.mean(condpred[1:,], axis=1), c="C1", zorder=3)
    ax.fill_between(range(steps, steps+h), *np.quantile(condpred[1:,], [0.05, 0.95], axis=1), color="C1", alpha=0.2)
    ax.plot(range(steps, steps+h), Yh, c="C2", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_title("Mackey-Glass | Autonomous ESN")
    ax.legend(["$x_t$", "$x_t - \epsilon_t$", "ESN Fit", "Best Cond Pred"], loc="center left")
    fig.canvas.draw()

interact(esn_nonlinear_ar, steps=(50, 500), h=(3, 100), Dx=(5,50), rho=(0.1, 1.5), gamma=(0.1,5), theta=(0.0, 1.0), alpha=(0.,1.),
            Lambda=widgets.FloatLogSlider(min=-9, max=2, value=1e-9));